**Bert model op basis van Bert van Universiteit van Groningen. Context handling moet nog aangepast worden. Oversampling a.d.h.v de mediaan. dynamisch treshhold zoeken voor unknown.**

In [1]:
import pandas as pd
import re
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch.nn.functional as F
import os 

In [2]:
# ✅ 1. Load & Preprocess Data
script_dir = os.path.dirname(os.getcwd()) # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")

# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data_cleaned.xlsx")
df = pd.read_excel(file_path)

In [3]:
import re

# Drop unnecessary columns
columns_to_drop = ["TXT_file_name", "context"]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# Handle missing values
df = df.dropna(subset=["question"])

# Clean text
def clean_text(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\b[a-z]\)\s+', ' ', text)
    text = re.sub(r'\b\d+\.\b', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply cleaning to just the question
df["clean_text"] = df["question"].apply(clean_text)

# Group by 'clean_text' and count unique themes
duplicates_with_diff_themes = df.groupby("clean_text")["theme"].nunique().reset_index()

# Filter rows where the number of unique themes is greater than 1
duplicates_with_diff_themes = duplicates_with_diff_themes[duplicates_with_diff_themes["theme"] > 1]

# Exclude rows with duplicate clean_texts that map to multiple themes
df = df[~df["clean_text"].isin(duplicates_with_diff_themes["clean_text"])]

# ✅ Drop rare themes (appearing < 2 times)
theme_counts = df["theme"].value_counts()
valid_themes = theme_counts[theme_counts >= 2].index
df = df[df["theme"].isin(valid_themes)]

# ✅ Recompute label encoding AFTER filtering
unique_themes = list(df["theme"].unique())
theme_to_id = {theme: idx for idx, theme in enumerate(unique_themes)}
id_to_theme = {idx: theme for theme, idx in theme_to_id.items()}
df["theme_id"] = df["theme"].map(theme_to_id)

# Amount of rows
print(f"Number of rows after filtering: {len(df)}")



Number of rows after filtering: 35194


In [4]:
print("All theme_ids:", sorted(df["theme_id"].unique()))
print("num_labels:", df["theme_id"].nunique())


All theme_ids: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
num_labels: 41


In [5]:
# ✅ 5. Split Data into Train & Test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["clean_text"].tolist(), df["theme_id"].tolist(), test_size=0.2, random_state=42, stratify=df["theme_id"]
)

In [8]:
from imblearn.over_sampling import RandomOverSampler
import pandas as pd

# Create a small DataFrame from train lists
train_df = pd.DataFrame({
    "clean_text": train_texts,
    "theme_id": train_labels
})

# Compute class counts and use median as balancing target
theme_counts = train_df["theme_id"].value_counts()
median_count = theme_counts.median()



from collections import Counter
print("Class distribution after oversampling:", Counter(train_labels))


Class distribution after oversampling: Counter({3: 7275, 25: 2163, 9: 2122, 16: 1798, 19: 1639, 1: 1458, 0: 1169, 2: 1022, 18: 1018, 6: 854, 26: 738, 17: 714, 22: 639, 10: 634, 21: 553, 27: 514, 7: 505, 23: 391, 8: 366, 24: 327, 14: 290, 12: 271, 13: 248, 35: 247, 11: 161, 32: 159, 28: 154, 15: 144, 33: 123, 4: 87, 36: 86, 5: 74, 30: 60, 20: 59, 31: 31, 29: 29, 40: 10, 38: 9, 39: 6, 34: 5, 37: 3})


In [9]:
# ✅ 7. Load BERT Tokenizer & Define Dataset Class
model_name = "bert-base-multilingual-cased"  # mBERT
tokenizer = BertTokenizer.from_pretrained(model_name)

class ThemeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx], padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt"
        )
        encoding["labels"] = torch.tensor(self.labels[idx])
        return {key: val.squeeze(0) for key, val in encoding.items()}

train_dataset = ThemeDataset(train_texts, train_labels, tokenizer)
test_dataset = ThemeDataset(test_texts, test_labels, tokenizer)

In [10]:
# ✅ 8. Load BERT Model for Classification
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=df["theme_id"].nunique())

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from sklearn.utils.class_weight import compute_class_weight
import torch

# Assuming train_labels is a list of class indices
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = torch.tensor(class_weights, dtype=torch.float)


In [14]:
from transformers import Trainer
from torch.nn import CrossEntropyLoss

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss


In [15]:
# ✅ 9. Define Training Arguments (With Early Stopping)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,                 # 👈 Keep only the last checkpoint
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,  
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",  # 🔥 Log only once per epoch
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)

# ✅ 10. Define Metrics for Evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ 11. Train Model with Early Stopping
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop if no improvement for 2 epochs
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.964900,2.378311,0.461003,0.590420,0.461003,0.485689
2,2.058700,2.060864,0.562296,0.613686,0.562296,0.568450
3,1.569700,1.864387,0.593692,0.647675,0.593692,0.605010
4,1.192800,1.779532,0.627788,0.666573,0.627788,0.638784
5,0.873600,1.827949,0.645830,0.681249,0.645830,0.655047
6,0.620400,1.936265,0.667993,0.680126,0.667993,0.670713
7,0.449000,1.984133,0.681631,0.682838,0.681631,0.680381
8,0.336800,2.025530,0.685467,0.690068,0.685467,0.686603


TrainOutput(global_step=28160, training_loss=1.2582311716946688, metrics={'train_runtime': 6961.6353, 'train_samples_per_second': 32.354, 'train_steps_per_second': 4.045, 'total_flos': 5.928388600025088e+16, 'train_loss': 1.2582311716946688, 'epoch': 8.0})

SAVE MODEL NAAR KUL DRIVE 

In [16]:
trainer.save_model("output/weighted-bert-model")


In [19]:
# Get predictions from the model
preds_output = trainer.predict(test_dataset)

# Extract logits (raw model outputs)
logits = preds_output.predictions  # shape: (num_samples, num_classes)

# Get predicted class labels
predictions = np.argmax(logits, axis=1)

# Save everything in a DataFrame
output_df = pd.DataFrame({
    "text": test_texts,
    "true_label": test_labels,
    "predicted_label": predictions,
    "logits": logits.tolist()  # Save logits for softmax + threshold later
})

# Save to CSV
output_df.to_csv("output/test_predictions.csv", index=False)


In [ ]:
from datetime import datetime

run_id = datetime.now().strftime("%Y-%m-%d_%H-%M")
#save_path = f"C:/Users/corne/OneDrive - KU Leuven/Thesis/Working Code/SAVED-Models/mBERT/Run_{run_id}"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)